###  Loading data for homework-04

In [2]:
import pandas as pd

github_url= 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv'

url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [4]:
df =  df.iloc[:300]

### Q1. Getting the embeddings model

In [5]:
from sentence_transformers import SentenceTransformer

model_name='multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/Users/zaahirdawood/Documents/llm-zoomcamp/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/zaahirdawood/Documents/llm-zoomcamp/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
answer_llm = df.iloc[0].answer_llm

embedding_model.encode(answer_llm)